In [47]:
#Importing necessary libraries

import selenium
from selenium import webdriver
import pandas as pd 
import numpy as np

from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException,WebDriverException, ElementClickInterceptedException, TimeoutException,ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup as bs

import time

from IPython.display import HTML
import pyshorteners as short

import warnings 
warnings.filterwarnings("ignore")

In [48]:
# Q1 Solution:

# defining a function to visit amazon website and search for a product.
def initiate_amazon(element):
    
    # mentioning the PIN Code of our location.
    PIN= "781028"
    # opening the target website.
    driver.get("https://www.amazon.in/")
    # putting the program on hold for 2 seconds so that the website gets loaded properly.
    time.sleep(2)
    
    # Using the explicit wait method till the location-button is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/header/div/div[1]/div[1]/div[2]/span/a/div[2]")))
    # finding the webelement for the location-button using absolute-xpath and clicking on it.
    location_btn= driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[1]/div[2]/span/a/div[2]").click()
    
    # putting the program on hold for 1 seconds so that the loation entry field is loaded properly.
    time.sleep(1)
    
    # Using the explicit wait method till the PIN entry field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//input[@id='GLUXZipUpdateInput']")))
    # finding the webelement for the PIN entry field using relative-xpath and sending the mentioned PIN.
    send_pin= driver.find_element(By.XPATH,"//input[@id='GLUXZipUpdateInput']").send_keys(PIN)
                   
    # clicking the apply button.
    apply_pin= driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div/div/div[2]/div[3]/div[2]/div/div[2]/span/span/input").click()
    
    #putting the program on hold for 2 seconds
    time.sleep(2)
    
    # Using the explicit wait method till the search field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")))
    # finding the webelement for the search field using absolute-xpath and sending the user-input data.
    srch= driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
    srch.send_keys(element)             
    
    # finding the webelement for the search button using absolute-xpath and clicking on it.
    try: 
        srch_button= driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input")
        srch_button.click()
    except WebDriverException:
        pass

# taking input from the user.
element= input("Search in Amazon: ")

# connecting to the webdriver.
driver= webdriver.Chrome("chromedriver.exe")

# calling the function above and sending the user-input as an arguement.
initiate_amazon(element)

Search in Amazon: TV


In [49]:
# Q2 Solution:

# defining the lists to be used.
brands=[]
names=[]
prices=[]
exchange_check=[]
return_duration=[]
exp_delvry=[]
availability=[]
url=[]

# defining a 'User-Agent' and 'Accept-Language' so that our requests don't get blocked.
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
            'Accept-Language': 'en-US, en;q=0.5'})

# an alternate.
# HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
#            'Accept-Language': 'en-US, en;q=0.5'})


# defining a function to get the price data.
def get_price(components):
    for j in components:
        check= j.find("₹")
        if check!=(-1):
            prices.append(j.split("₹")[1])
            break
        elif j==components[-1]:
            prices.append("-")
        else:
            continue
            
# defining a function to get the availability data.
# amazon by default only lists products which are available.
# so we only get those which are in short supply and running out fast.
def availability_checker(components):
    availability_check= components.text.find("left in stock")
    if availability_check==(-1):
        availability.append("Available")
    else:
        availability.append(components.text.split("\n")[-1])

# defining a function to get the expected delivery data.
def delivery(components):
    for j in components:
        check1= j.find("Get it by")
        check2= j.find("FREE delivery by")
        if check1!=(-1): 
            exp_delvry.append(j.split("Get it by")[-1])
            return 1
        elif check2!=(-1):
            exp_delvry.append(j.split("FREE delivery by")[-1])
            return 1
        elif j==components[-1]:
            return 0
        else:
            continue

# defining a function to get the exchange, return and delivery(if delivery data is missing is search-results page) data.
# It takes in two arguements. One is a link to a product and another is status of expected-delivery data.
def check_return_exchange(link,status):
    
    # sending request to the target website.
    r= requests.get(link,headers=HEADERS)
    
    # getting the content in the form of a beautifulsoup object from which we will extract our data. 
    soup= bs(r.content)
    
    # getting the exchange data using tag name and id. If the element is absent then exchange is "Not Applicable"
    # if the element is present then either there is an exchange-deal or exchange is not avavilable at our geographical location.
    # data is filled accordingly.
    try:
        exchange_data=soup.find("div",id="buyBackAccordionRow").text.strip()
        check= exchange_data.find("Up to")
        if check!=(-1):
            upto_value= exchange_data.split("off")[0].strip().split(" ")[-1]
            exchange_check.append(f'Yes, Upto Rs.{upto_value} off')
        else:
            exchange_check.append("Not Available at your location")
    except AttributeError:
        exchange_check.append("Not Applicable")
    
    # getting the return-policy data using tag name and id. If the element is absent then it is "Not Available".
    try:
        return_data= soup.find("div",id="RETURNS_POLICY").text.strip()
        return_duration.append(return_data)
    except AttributeError:
        return_duration.append("Not Available")
    
    # status=0 if we didn't find our delivery data in product-listing page. Hence we extract it from the product page itself.
    # If the data is absent here too then we append null.
    try:
        if status==0:
            delivery_data= soup.find("div",id='deliveryBlockMessage').text.strip()
            exp_delvry.append(delivery_data.split("FREE delivery")[1].strip().split(".")[0])
        else:
            pass           
    except AttributeError:
        exp_delvry.append("-")
    except IndexError:
        exp_delvry.append("+")

        
# the pages that shows the listing of products are basically of 3 types based on class values.
# Hence defining functions for each of these types to get data.

# defining a function to get the data from one type of product-listing page.
def get_data_type1():
    
    # using the explicit wait method till the webelements we want gets loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='a-section a-spacing-base']")))
    # finding the webelements for the products listed on the page.
    data=driver.find_elements(By.XPATH,"//div[@class='a-section a-spacing-base']")

    # running a for loop to process each product data and extract the relevant information.
    for i in data:
        # pre-processing the text-data to remove some irrelevant data and get relevant ones in a list for further processing.
        info= i.text.split("Sponsored")[-1].strip().split("Amazon's Choice")[-1].strip().split("Best seller")[-1].strip()
        if info=="":
            continue
        else:
            elements= info.split("\n")
            
            if elements[0][0]=="+":
                elements.pop(0)
            else: 
                pass
            
            # extracting the relevant data into respective lists.
            brands.append(elements[0].split(" ")[0])
            names.append(elements[0])
            
            get_price(elements)         # calling the "get_price" function to get the price data.
            availability_checker(i)     # calling the "availability_checker" function to get the availability data.          
            status=delivery(elements)   # the "delivery" function will return 0 if it can't find the expected-delivery data on the product listing page, otherwise it will return 1.
            
            # getting the link, saving it in a list and send the link to "check_return_exchange" function to be processed for further information.
            link_data= i.find_element(By.XPATH,".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
            link=link_data.get_attribute('href')
            url.append(link)
            check_return_exchange(link,status)

# defining a function to get the data from another type of product-listing page.
def get_data_type2():
    
    # using the explicit wait method till the webelements we want gets loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='a-section a-spacing-base a-text-center']")))
    # finding the webelements for the products listed on the page.
    data=driver.find_elements(By.XPATH,"//div[@class='a-section a-spacing-base a-text-center']")
    
    # running a for loop to process each product data and extract the relevant information.
    for i in data:
        # pre-processing the text-data to remove some irrelevant data and get relevant ones in a list for further processing.
        info= i.text.split("Sponsored")[-1].strip().split("Amazon's Choice")[-1].strip().split("Best seller")[-1].strip()
        if info=="":
            continue
        else:
            elements= info.split("\n")
            
            if elements[0][0]=="+":
                elements.pop(0)
            else:
                pass
            
            # extracting the relevant data into respective lists.
            brands.append(elements[0])
            names.append(elements[1])
            
            get_price(elements)            # calling the "get_price" function to get the price data.           
            availability_checker(i)        # calling the "availability_checker" function to get the availability data.                           
            status=delivery(elements)      # the "delivery" function will return 0 if it can't find the expected-delivery data on the product listing page, otherwise it will return 1.
            
            # getting the link, saving it in a list and send the link to "check_return_exchange" function to be processed for further information.
            link_data= i.find_element(By.XPATH,".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
            link=link_data.get_attribute('href')
            url.append(link)
            check_return_exchange(link,status)

# defining a function to get the data from another type of product-listing page.
def get_data_type3():
    
    # using the explicit wait method till the webelements we want gets loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='a-section']")))
    # finding the webelements for the products listed on the page.
    data=driver.find_elements(By.XPATH,"//div[@class='a-section']")
    
    # running a for loop to process each product data and extract the relevant information.
    for i in data:
        # pre-processing the text-data to remove some irrelevant data and get relevant ones in a list for further processing.
        info= i.text.split("Sponsored")[-1].strip().split("Amazon's Choice")[-1].strip().split("Best seller")[-1].strip()
        if (info=="")or(i==data[-1]):
            continue
        else:
            elements= info.split("\n")
            
            if elements[0][0]=="+":
                elements.pop(0)
            else:
                pass
            
            # extracting the relevant data into respective lists.
            brands.append(elements[0].split(" ")[0])
            names.append(elements[0])
            
            get_price(elements)            # calling the "get_price" function to get the price data.               
            availability_checker(i)        # calling the "availability_checker" function to get the availability data.          
            status=delivery(elements)      # the "delivery" function will return 0 if it can't find the expected-delivery data on the product listing page, otherwise it will return 1. 
            
            # getting the link, saving it in a list and send the link to "check_return_exchange" function to be processed for further information.
            link_data= i.find_element(By.XPATH,".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
            link=link_data.get_attribute('href')
            url.append(link)
            check_return_exchange(link,status)

# defining the function to start the data-scraping process.            
def amazon_scrapper():
    
    # putting the program on hold for 2 seconnds so that the search-results page gets loaded properly.
    time.sleep(2)
    
    # finding the webelements for the 3 types of product listings. 
    data_type1=driver.find_elements(By.XPATH,"//div[@class='a-section a-spacing-base']")
    data_type2=driver.find_elements(By.XPATH,"//div[@class='a-section a-spacing-base a-text-center']")
    data_type3=driver.find_elements(By.XPATH,"//div[@class='a-section']")
    # saving the number of webelements found for each type in a list.
    data_lengths=[len(data_type1),len(data_type2),len(data_type3)]

    # getting the webelements for the page2 and 3 buttons and extracting the links into a list.
    pages_btn_data= driver.find_elements(By.XPATH,"//a[@class='s-pagination-item s-pagination-button']")
    pages=[]
    for page in pages_btn_data:
        pages.append(page.get_attribute('href'))
    
    # defining a variable to keep track of the page getting processed.
    count=1
    
    # finding the webelements' type which have the highest count using if-else conditions. That is our product-listings of the page.
    if (data_lengths[0]>data_lengths[1])&(data_lengths[0]>data_lengths[2]):
        print(f'Processing page{count}...')
        get_data_type1()         # calling the function to scrape type1 data.
        for page in pages:
            count+=1
            print(f'Processing page{count}...')
            driver.get(page)     # loading the next products-listing page.
            time.sleep(1)            
            get_data_type1()     # calling the function to scrape type1 data.

    elif (data_lengths[1]>data_lengths[0])&(data_lengths[1]>data_lengths[2]):
        print(f'Processing page{count}...')
        get_data_type2()         # calling the function to scrape type2 data.
        for page in pages:
            count+=1
            print(f'Processing page{count}...')
            driver.get(page)     # loading the next products-listing page.  
            time.sleep(1)            
            get_data_type2()     # calling the function to scrape type2 data.
            
    elif (data_lengths[2]>data_lengths[0])&(data_lengths[2]>data_lengths[1]):
        print(f'Processing page{count}...')
        get_data_type3()         # calling the function to scrape type3 data.
        for page in pages:
            count+=1
            print(f'Processing page{count}...')
            driver.get(page)     # loading the next products-listing page.
            time.sleep(1)            
            get_data_type3()     # calling the function to scrape type3 data.

    else:
        pass
    
    # preparing the final dataframe.
    products= pd.DataFrame({"Brand":brands,
                            "Product Name":names, 
                            "Price(₹)":prices, 
                            "Availability":availability, 
                            "Expected Delivery":exp_delvry,
                            "Return Policy":return_duration,
                            "Exchange":exchange_check,
                            "Product Url":url
                            })
    # returning the dataframe.
    return products

# calling the "amazon_scrapper" function to start the scraping process and save the result.
products_df= amazon_scrapper()

# closing the webdriver.
driver.close()

# saving the results in a csv file.
products_df.to_csv(f'{element}_search_results.csv',header=True)
products_df

Processing page1...
Processing page2...
Processing page3...


,Brand,Product Name,Price(₹),Availability,Expected Delivery,Return Policy,Exchange,Product Url
0,AmazonBasics,AmazonBasics 81 cm (32 inches) HD Ready Smart ...,"13,999",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/gp/slredirect/picassoRed...
1,TCL,TCL 100 cm (40 inches) Full HD Certified Andro...,"19,990",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/gp/slredirect/picassoRed...
2,OnePlus,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,"14,999",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/OnePlus-inches-Ready-And...
3,Redmi,Redmi 80 cm (32 inches) Android 11 Series HD R...,"14,999",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/Redmi-inches-Ready-L32M6...
4,Mi,Mi 80 cm (32 inches) 5A Series HD Ready Smart ...,"15,499",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/inches-Ready-Smart-Andro...
...,...,...,...,...,...,...,...,...
61,LG,LG 108 cm (43 inches) 4K Ultra HD Smart LED TV...,"31,490",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/LG-inches-Ultra-43UQ7500...
62,Vu,Vu 108 cm (43 Inches) Premium 4K Series Smart ...,"25,999",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/Inches-Premium-Smart-And...
63,Toshiba,Toshiba 80 cm (32 inches) V Series HD Ready Sm...,"14,990",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/Toshiba-inches-Android-3...
64,Hyundai,Hyundai 80 cm (32 inches) HD Ready Smart LED T...,"13,490",Available,"Wed, 3 Aug, 7:00 am - 9:00 pm",10 Days Replacement,"Yes, Upto Rs.2,050.00 off",https://www.amazon.in/gp/slredirect/picassoRed...


In [50]:
# Q3 Solution:

# defining a function to scrap images from google-images.
def google_image_scrapper(keyword):
    
    # connecting to the webdriver.
    driver= webdriver.Chrome("chromedriver.exe")
    # opening the target website. 
    driver.get("https://images.google.com/")
    # maximizing the driver window.
    driver.maximize_window()
    # refreshing the website.
    driver.refresh()
    
    # using the explicit wait method till the search field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")))
    # finding the webelement for the search field using absolute-xpath and sending the required keys.
    srch_bar= driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
    srch_bar.send_keys(keyword)
    
    # finding the webelement for the search button using absolute-xpath and clicking on it.
    srch_btn= driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button/div/span").click()
    
    # putting the program on hold for 2 seconds so that the search is processed and the page is loaded properly.
    time.sleep(2)
    
    # running a for loop for each webelement of the images and taking a screenshot of them and saving them in a specific directory.
    # the xpath of each of the images vary only by the value of X in "//*[@id="islrg"]/div[1]/div[X]/a[1]/div[1]/img"
    for i in range(1, 11):
        try:
            driver.find_element(By.XPATH,'//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').screenshot(f'C:\\Users\\Asus\Pictures\\Saved Pictures\\{keyword} ('+str(i)+').png')
        except:
            pass
        
    # closing the webdriver
    driver.close()
    
# taking input from the user.        
keyword= input("Search in Google-Images: ")
# calling the function and sending the user-input as an arguement.
google_image_scrapper(keyword)

Search in Google-Images: cakes


In [29]:
# Q4 Solution:

# defining the lists to be used.
brand=[]
name=[]
color=[]
ram=[]
rom=[]
p_cam=[]
s_cam=[]
disp=[]
battery=[]
price=[]
url=[]

# defining the function for scraping smartphones from flipkart.com
def flipkart_smartphone_scrapper(element):
    
    # connecting to the webdriver.
    driver= webdriver.Chrome("chromedriver.exe")
    # opening the target website. 
    driver.get("https://www.flipkart.com")
    
    # using the explicit wait method till the close-button in the login form is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[2]/div/div/button")))
    # finding the webelement for the close-button in the login form using absolute-xpath and clicking on it.
    button= driver.find_element(By.XPATH,"/html/body/div[2]/div/div/button")
    button.click()
    
    # using the explicit wait method till the search-field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input")))
    # finding the webelement for the search field using absolute-xpath and sending the required keys.
    srch_sunglasses= driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input")
    srch_sunglasses.send_keys(element)

    # finding the webelement for the search button using relative-xpath and clicking on it.
    search_btn= driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']")
    search_btn.click()

    # putting the program on hold for 5 seconds.
    time.sleep(5)

    # Using the explicit wait method till the elements we want is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='_4rR01T']")))

    # getting the webelements containing brand,product-name and color data using relative-xpath and then extract the text-data which is processed using split-method and then put them into corresponding lists. 
    brand_data= driver.find_elements(By.XPATH,"//div[@class='_4rR01T']")
    for i in brand_data:
        brand.append(i.text.split(" ")[0])
        name.append(i.text.split("(")[0])
        color.append(i.text.split("(")[1].split(",")[0])

    # getting the webelements containing specifications data using relative-xpath and then extract the text-data which is processed using split-method and then put them into corresponding lists. 
    spec_data= driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']")
    for i in spec_data:
        ram.append(i.text.split("\n")[0].split("|")[0])
        rom.append(i.text.split("\n")[0].split("|")[1])
        disp.append(i.text.split("\n")[1])
        battery.append(i.text.split("\n")[3])

        contents= i.text.split("\n")[2].split("|")
        if len(contents)==1:
            p_cam.append(contents)
            s_cam.append("-")
        else :
            p_cam.append(contents[0])
            s_cam.append(contents[1])

    # getting the webelements containing url data using relative-xpath and then extract the links using the 'href' attribute into a list.
    url_data= driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
    for i in url_data:
        url.append(i.get_attribute('href'))

    # getting the webelements containing price data using relative-xpath and then extract the text-data into a list.
    price_data= driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
    for i in price_data:
        price.append(i.text)

    # the final dataframe
    results= pd.DataFrame({"Brand":brand,
                           "Product Name":name, 
                           "Color":color, 
                           "RAM":ram, 
                           "ROM":rom, 
                           "Primary Camera":p_cam,
                           "Secondary Camera":s_cam,
                           "Display":disp,
                           "Battery":battery,
                           "Price":price,
                           "url":url})
    
    # returning the final dataframe. 
    return results
    # closing the webdriver.
    driver.close()

# taking input from the user.
element= input("Search a smartphone: ")

# calling the scrapper function and sending the user input as an arguement. 
results= flipkart_smartphone_scrapper(element)

# exporting to a csv file in current directory.
results.to_csv(f'{element} _search_results.csv',header=True,index=False)

results

Search a smartphone: asus zenfone


,Brand,Product Name,Color,RAM,ROM,Primary Camera,Secondary Camera,Display,Battery,Price,url
0,ASUS,ASUS ZenFone Max M2,Black,3 GB RAM,32 GB ROM,13MP + 2MP,8MP Front Camera,15.9 cm (6.26 inch) HD+ Display,4000 mAh Battery,"₹12,549",https://www.flipkart.com/asus-zenfone-max-m2-b...
1,ASUS,ASUS ZenFone Max Pro M2,Blue,3 GB RAM,32 GB ROM,12MP + 5MP,13MP Front Camera,15.9 cm (6.26 inch) Full HD+ Display,5000 mAh Battery,"₹15,999",https://www.flipkart.com/asus-zenfone-max-pro-...
2,ASUS,ASUS ZenFone Max M2,Black,4 GB RAM,64 GB ROM,13MP + 2MP,8MP Front Camera,15.9 cm (6.26 inch) HD+ Display,4000 mAh Battery,"₹14,999",https://www.flipkart.com/asus-zenfone-max-m2-b...
3,ASUS,ASUS ZenFone 5Z,Meteor Silver,8 GB RAM,256 GB ROM,12MP + 8MP,8MP Front Camera,15.75 cm (6.2 inch) Full HD+ Display,3300 mAh Battery,"₹43,999",https://www.flipkart.com/asus-zenfone-5z-meteo...
4,ASUS,ASUS Zenfone 4 Selfie,Black,3 GB RAM,32 GB ROM,13MP Rear Camera,13MP Front Camera,13.97 cm (5.5 inch) HD Display,3000 mAh Battery,"₹10,999",https://www.flipkart.com/asus-zenfone-4-selfie...
5,ASUS,ASUS Zenfone Max Pro M1,Grey,6 GB RAM,64 GB ROM,16MP + 5MP,16MP Front Camera,15.21 cm (5.99 inch) Full HD+ Display,5000 mAh Battery,"₹17,999",https://www.flipkart.com/asus-zenfone-max-pro-...
6,ASUS,ASUS ZenFone Max M2,Blue,3 GB RAM,32 GB ROM,13MP + 2MP,8MP Front Camera,15.9 cm (6.26 inch) HD+ Display,4000 mAh Battery,"₹12,549",https://www.flipkart.com/asus-zenfone-max-m2-b...
7,ASUS,ASUS Zenfone Max Pro M1,Blue,4 GB RAM,64 GB ROM,13MP + 5MP,8MP Front Camera,15.21 cm (5.99 inch) Full HD+ Display,5000 mAh Battery,"₹15,599",https://www.flipkart.com/asus-zenfone-max-pro-...
8,ASUS,ASUS Zenfone Max Pro M1,Grey,4 GB RAM,64 GB ROM,13MP + 5MP,8MP Front Camera,15.21 cm (5.99 inch) Full HD+ Display,5000 mAh Battery,"₹15,599",https://www.flipkart.com/asus-zenfone-max-pro-...
9,ASUS,ASUS Zenfone Max Pro M1,Blue,6 GB RAM,64 GB ROM,16MP + 5MP,16MP Front Camera,15.21 cm (5.99 inch) Full HD+ Display,5000 mAh Battery,"₹17,999",https://www.flipkart.com/asus-zenfone-max-pro-...


In [30]:
# Q5 solution:

# defining a function to get the coordinates from google-maps.
def city_coordinates(element):
    # connecting to the webdriver.
    driver= webdriver.Chrome("chromedriver.exe")
    # opening the target website. 
    driver.get("https://maps.google.com")

    # putting the program on hold for 3 seconds so that the page gets loaded.
    time.sleep(3)
    
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]")))
    # finding the webelement for the search field using absolute-xpath and sending the required keys.
    srch_city= driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]")
    srch_city.send_keys(element)

    # finding the webelement for the search button using absolute-xpath and clicking on it.
    srch_btn= driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
    srch_btn.click()

    # putting the program on hold for 4 seconds so that the page gets loaded.
    time.sleep(4)

    # getting the "current-url" text and extracting the coordinates values using split method.
    contents= driver.current_url.split("@")[1].split("/")[0].split(",")

    # print the coordinates.
    print("Geospatial coordinates: ",contents[0],",",contents[1])

    #closing the webdriver
    driver.close()
    
# taking input from the user.
element= input("Enter a city name: ")
# calling the function and sending the user input as an arguement. 
city_coordinates(element)

Enter a city name: new delhi
Geospatial coordinates:  28.527582 , 77.0688958


In [31]:
# Q6 solution:

# defining a list of lists to be used.
lists=[[],[],[],[],[],[],[],[]]

# defining a function which takes 3 arguements- column-number, a list and a beautifulsoup object.
# this function will extract the text data in the column to a list.
def get_data(column_number,list_object,soup):
    data= soup.find_all("td",class_=f'column-{column_number}')
    for j in data:
        list_object.append(j.text)
        
# defining a function to scrap data from the website.
def scrap_trak(url):
    # sending request to the target website.
    r= requests.get(url)
    # getting the parsed content from which we will extract our data. 
    soup= bs(r.content)  

    # using a variable named "count" to send the correct column-number to the function defined above.
    count=2

    # filling each list in the lists.
    for i in lists:
        get_data(count,i,soup)
        count+=1

    # making a dataframe from the list of lists.
    x= pd.DataFrame(lists)
    # taking a transpose of the dataframe since our desired columns are the rows in our dataframe.
    df= x.T  

    # naming the columns.
    df.columns=["Date","Startup Name","Industry/Vertical","Sub-Vertical","City/Location","Investor's Name","Investment Type","Amount(In USD)"]  

    # cleaning an entry so that "to-datetime" method can be applied on the "Date" column.
    df["Date"].replace("05/072018","05/07/2018", inplace=True)

    # using the pandas "to_datetime" method to convert the text data in "Date" column to Date-Time format.
    df["Date"]= pd.to_datetime(df["Date"], dayfirst=True)
    
    #returning the resultant dataframe.
    return df

# the url from which we will scrap the data.
url= "https://trak.in/india-startup-funding-investment-2015/"

# calling the "scrap_trak" function with the url as arguement.
df= scrap_trak(url)

# extracting the data for Jan 2021 – March 2021.
final_df= df[(df['Date'] >= '2021-01-01') & (df['Date'] <= '2021-03-31')]
# reset the index of final dataframe.
final_df.reset_index(drop=True,inplace=True)

final_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investor's Name,Investment Type,Amount(In USD)
0,2021-03-04,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,2021-03-31,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,2021-03-30,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,2021-03-30,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,2021-03-23,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,2021-03-25,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,2021-03-26,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
7,2021-02-11,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
8,2021-02-22,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
9,2021-02-19,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"


In [32]:
# Q7 Solution:

# defining the lists to be used.
names=[]
processor=[]
display=[]
os=[]
memory=[]
graphics=[]
body=[]
prices=[]
links=[]

# defining a function to scrap gaming-laptops data from digit.in
def gaming_laptops(url):
    
    # sending request to the target website.
    r= requests.get(url)
    # getting the parsed content from which we will extract our data. 
    soup= bs(r.content)

    # getting the elements having the names and extracting the text into a list. 
    data= soup.find_all("h3")
    for i in data:
        names.append(i.text)

    # getting the elements having the links to each product and extracting them into a list.   
    data=soup.find_all("div",id=["toptenIdevent1","toptenIdevent2","toptenIdevent3","toptenIdevent4","toptenIdevent5","toptenIdevent6","toptenIdevent7","toptenIdevent8","toptenIdevent9","toptenIdevent10"])
    for i in data:
        link= i.find("a",href=True)
        links.append(link['href'])

    # getting the elements having the specifications and price data and extracting the relevant text into the corresponding lists. 
    data= soup.find_all("div",class_="Section-center")
    count=0
    for i in data:
        spec_data=i.find("div",class_="Spcs-details")
        x= spec_data.text.strip().split("\n")
        processor.append(x[4])
        display.append(x[9])
        os.append(x[14])
        memory.append(x[19])
        graphics.append(x[24])
        body.append(x[29])

        price_data= i.find("div",class_="merchantdatalist1 merchantdatalist2").text.strip()
        if price_data=="":
            # the price is not mentioned in the main page for some products. Hence extracting that info from that product page.
            r_= requests.get(links[count])
            soup_=bs(r_.content)
            content=soup_.find("div",class_="price").text.strip().split("\t")
            if content[-1]=="":
                prices.append("--")
            else:
                prices.append(content[-1])
        else:
            prices.append(price_data.split("₹")[1].strip().split(" ")[0])
        count+=1  

    # the final dataframe
    gmng_laptops= pd.DataFrame({"Product Name":names,
                                "Processor":processor,
                                "Display":display, 
                                "OS":os, 
                                "Memory":memory, 
                                "Graphics":graphics, 
                                "Body":body,
                                "Price(₹)":prices,
                                "Link": links                        
                                })
    # returning the final dataframe.
    return gmng_laptops

# the url from which we will scrap the data.
url= "https://www.digit.in/top-products/best-gaming-laptops-40.html"

# calling the "gaming_laptops" function with the url as arguement and saving the result.
df= gaming_laptops(url)
df

,Product Name,Processor,Display,OS,Memory,Graphics,Body,Price(₹),Link
0,MSI TITAN GT77-12UHS,12th Gen Intel Core i9-12900HX 16 core processor,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,--,https://www.digit.in/laptops/msi-titan-gt77-12...
1,ALIENWARE X17 R2,12th Gen Intel Core i9-12900HK 14 core process...,"17.3″ (1920x1080) screen, 360 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,"16 GB DDR6 NVIDIA GeForce RTX 3080 Ti, Graphic...",399.23 x 299.57 x 14.75 mm dimension & 3.02 kg...,"389,990",https://www.digit.in/laptops/alienware-x17-r2-...
2,ACER PREDATOR TRITON 500 SE PT516-52S,12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 240 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,358 x 262 x 19.9 mm dimension & 2.4 kg weight,"300,000",https://www.digit.in/laptops/acer-predator-tri...
3,OMEN BY HP (16-B1371TX),12th Gen Intel Core i7-12700H 14 core processo...,16.1″ (2560 x 1440) screen,Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,"195,600",https://www.digit.in/laptops/omen-by-hp-16-b13...
4,ACER PREDATOR HELIOS 300 AN515-45 (NH.QBRSI.0,AMD Ryzen 9-5900HX 8 core processor,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,"172,999",https://www.digit.in/laptops/acer-nitro-5-an51...
5,MSI DELTA 15 (A5EFK-083IN),AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,"188,990",https://www.digit.in/laptops/msi-delta-15-5th-...
6,OMEN BY HP (16-C0141AX),AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16.1″ (2560 x 1440) screen,Windows 11 Home,16 GB ‎DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,"129,899",https://www.digit.in/laptops/hp-omen-16-c0141a...
7,LENOVO LEGION 5I PRO (82RF00MGIN),12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,359.9 x 264.4 x 19.9 mm dimension & 2.5 kg weight,"230,890",https://www.digit.in/laptops/lenovo-legion-5i-...
8,ALIENWARE M15 R5 RYZEN EDITION ICC-C780001WIN,AMD Ryzen R7-5800H 8 core processor with 4.40 ...,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,"140,999",https://www.digit.in/laptops/dell-alienware-m1...
9,LENOVO SLIM 7 GEN 6 (82K8002JIN),Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,"15.6″ screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,"135,490",https://www.digit.in/laptops/lenovo-legion-s7-...


In [33]:
# Q8 Solution:

# defining the lists to be used.
ranks=[]
names=[]
net_wrth=[]
age=[]
ctznsp=[]
source=[]
industry=[]
count=0

# defining a function to append data into our lists.
def fill_data(content):
    ranks.append(content[0])
    names.append(content[1])
    net_wrth.append(content[2])
    age.append(content[3])
    ctznsp.append(content[4])
    source.append(content[5])
    industry.append(content[6])

# defining a function to append null values incase we face "WebDriverException".
# the "count" variable can be used to check if we had any missing entries.
def fill_na():
    ranks.append("-")
    names.append("-")
    net_wrth.append("-")
    age.append("-")
    ctznsp.append("-")
    source.append("-")
    industry.append("-")
    count+=1
    
# defining a function to scrap data.
def billionaires_data(url):
    # connecting to the webdriver.
    driver= webdriver.Chrome("chromedriver.exe")
    # opening the target website.
    driver.get(url)

    # putting the program on hold for 5 seconds so that the website gets loaded properly.
    time.sleep(5)
    
    # using the explicit wait method till the menu button(three-dash button) gets loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/header/nav/div[1]/button[1]")))
    # finding the webelement for the menu button using absolute x-path and clicking on it.
    menu_btn= driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[1]/button[1]").click()
    
    # using the explicit wait method till the element we want gets loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")))
    # finding the webelement for the "World's Billionaires" option using absolute x-path and getting the link from it using the "href" attrubute.
    all_billnrs= driver.find_element(By.XPATH,"/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
    link= all_billnrs.get_attribute("href")

    # opening the "World's Billionaires" webpage.
    driver.get(link)
    # Using the explicit wait method till the element we want is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//button[@class='goto-page__btn']")))
    # finding the webelements for the buttons to different pages having the list of billionaires.
    btns= driver.find_elements(By.XPATH,"//button[@class='goto-page__btn']")

    # using a for loop to go through each page.
    for page in btns[1:]:
        # using the explicit wait method till the element we want is loaded.
        wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]")))

        # finding the webelement for the first entry in a page using absolute x-path and then processing it and extracting the text data.
        # If we face "WebDriverException" error then we just add "-" as the entry.
        list_top_data= driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]")
        try:
            content=list_top_data.text.strip().split("\n")
            fill_data(content)
        except WebDriverException:
            fill_na()

        # finding the webelements for all the entries in a page(except the first entry) using relative x-path and then processing it and extracting the text data.
        # If we face "WebDriverException" error then we just add "-" as the entry.
        data= driver.find_elements(By.XPATH,"//div[@class='table-row ']")
        for i in data:
            try:
                content=i.text.strip().split("\n")
                fill_data(content)
            except WebDriverException:
                fill_na()

        # clicking the button for next page and put the program on hold for 2 seconds so that the page gets loaded properly.
        page.click()
        time.sleep(2)

        # collecting the data for the last page.
        if page== btns[-1]:        
            list_top_data= driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]")
            try:
                content=list_top_data.text.strip().split("\n")
                fill_data(content)
            except WebDriverException:
                fill_na()

            data= driver.find_elements(By.XPATH,"//div[@class='table-row ']")
            for i in data:
                try:
                    content=i.text.strip().split("\n")
                    fill_data(content)
                except WebDriverException:
                    fill_na()

        else:
            continue 

    # the final dataframe
    billionaires= pd.DataFrame({"Rank":ranks,
                                "Name":names,
                                "Net Worth":net_wrth, 
                                "Age":age, 
                                "Citizenship":ctznsp, 
                                "Source":source, 
                                "Industry":industry                       
                               })
    # closing the webdriver
    driver.close()
    #returning the dataframe.
    return billionaires

# the url from which we will scrap the data.
url= "https://www.forbes.com/"

# calling the "billionaires_data" function with the url as arguement.
df= billionaires_data(url)
df

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,66,China,Fiberglass,Manufacturing
2664,2578.,Zhou Ruxin,$1 B,59,China,Navigation,Technology
2665,2578.,Wen Zhou & family,$1 B,57,China,chemicals,Manufacturing
2666,2578.,Zhou Yifeng & family,$1 B,43,China,liquefied petroleum gas,Energy


In [53]:
# Q9 Solution:

# defining a function to scrap youtube comments.
def youtube_comments(url):
    
    # connecting to the webdriver.
    driver= webdriver.Chrome("chromedriver.exe")
    # opening the target website.
    driver.get(url)
    # putting the program on hold for 5 seconds so that the webpage gets loaded properly.
    time.sleep(5)
    
    # we can use the "page-up","page-down","end",etc keyboard keys by using the "send_keys" method and "Keys" class of selenium.
    # since we have to scrap hundreds of comments hence running a loop of going to page-end with 3 seconds gap.
    html = driver.find_element(By.TAG_NAME,'html')
    for i in range(0,30):    
        html.send_keys(Keys.END)
        time.sleep(3)
    
    # Using the explicit wait method till the comments data is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//ytd-comment-renderer[@class='style-scope ytd-comment-thread-renderer']")))
    # finding the webelements having the comments data.
    data= driver.find_elements(By.XPATH,"//ytd-comment-renderer[@class='style-scope ytd-comment-thread-renderer']")
    
    #defining the lists to be used.
    comments=[]
    time_ago=[]
    upvotes=[]
    
    # running a for loop for each comment webelement and extracting the text data into corresponding lists.
    for i in data:
        content=i.text.strip().split("\n")
        upvote= i.text.split("REPLY")[0].strip().split("\n")[-1]
        comments.append(content[2])
        time_ago.append(content[1])

        if len(upvote)<=4:
            upvotes.append(upvote)
        else:
            upvotes.append(0)
    
    # the final dataframe.
    yt_comments= pd.DataFrame({"Comment":comments,
                                "Posted":time_ago,
                                "Upvotes":upvotes                        
                               })
    # returning the dataframe.
    return yt_comments

# the youtube url from which we want to scrap the comments data.
url= "https://www.youtube.com/watch?v=9bZkp7q19f0"

# calling the "youtube_comments" function with the url as arguement.
df= youtube_comments(url)
df

,Comment,Posted,Upvotes
0,"Lets be honest here, this guy owned the world ...",2 years ago (edited),142K
1,"This guy appeared out of nowhere, and everyone...",2 years ago,50K
2,Exatamente 10 anos dessa música. Lembro quando...,2 weeks ago,3.8K
3,Can't believe it's been 10 years since PSY inv...,2 weeks ago,909
4,Saudades dessa época! Aeee cadê os BR??? 🇧🇷🇧🇷🇧🇷,2 years ago,27K
...,...,...,...
575,Hace exactamente 10 años de esta gran canción,2 weeks ago,0
576,10 años de esta canción.. una locura.. recuerd...,2 weeks ago,12
577,Meu pai toda hora canta essa música por isso c...,2 weeks ago,8
578,"Despues de 10 años, lo vuelvo a escuchar esta ...",2 weeks ago,5


In [36]:
# Q10 Solution:

# defining a 'User-Agent' and 'Accept-Language' so that our requests don't get blocked.
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
# defining the lists to be used.
names=[]
dist_cc=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
pvt_prices=[]
dorm_prices=[]
facilities=[]
prop_des=[]

# defining a function to get the data from a hostel-listings page.
def get_data():
    
    # putting the program on hold for 2 seconds so that the search results gets loaded properly.
    time.sleep(2)
    
    # finding the webelements for the hostels shown.
    try:
        wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='title-row']")))
        data= driver.find_elements(By.XPATH,"//div[@class='title-row']")
    except TimeoutException:
        time.sleep(5)
        wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='title-row']")))
        data= driver.find_elements(By.XPATH,"//div[@class='title-row']")
    
    # checking the length of the data found and if there are "featured" properties(shown at top) listed then they are removed.
    # they are removed since "featured properties" are also listed in the all properties.
    if len(data)==33:
        data=data[3:]
    elif len(data)==32:
        data=data[2:]
    elif len(data)==31:
        data=data[1:]
    else:
        pass
    
    # getting the name and distance from city center data into a list.
    for i in data:
        names.append(i.text.split("\n")[0])
        dist_cc.append(i.text.split("\n")[1].split("-")[1].strip().split(" ")[0])
        
    # getting the dorm and privates prices data and appending them to respective lists.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='price-col']")))
    data= driver.find_elements(By.XPATH,"//div[@class='price-col']")
    for i in data[1::2]:
        dorm_prices.append(i.text.split("Rs")[-1])
    for i in data[0::2]:
        pvt_prices.append(i.text.split("Rs")[-1])

    # getting the links data and extracting the links to a list. 
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//a[@class='view-button']")))
    data= driver.find_elements(By.XPATH,"//a[@class='view-button']")
    links=[]
    for i in data:
        links.append(i.get_attribute("href"))
    
    # sending requests to each property link and getting the required info from them.
    for i in links:
        r=requests.get(i,headers=HEADERS)
        soup=bs(r.content)
        
        # getting the ratings, overall review and total-reviews data. In case of missing data "-" is appended to the respective list.
        try:
            ratings.append(soup.find("div",class_="score orange big").text)
            overall_reviews.append(soup.find("div",class_="keyword").text)
            total_reviews.append(soup.find("div",class_="reviews").text)
        except AttributeError:
            try:        
                ratings.append(soup.find("div",class_="score gray big").text)
                overall_reviews.append("-")
                total_reviews.append(soup.find("div",class_="reviews").text)
            except AttributeError:
                ratings.append("-")
                overall_reviews.append("-")
                try:              
                    total_reviews.append(soup.find("div",class_="reviews").text)
                except AttributeError:
                    total_reviews.append("-")
        
        # getting the property description data and extracting the text into a list.
        # incase of missing data "-" is appended.
        prop_data= soup.find_all("div",class_="description-container")
        try:
            if len(prop_data)==1:
                prop_des.append(prop_data[0].text.strip().split("Property Description")[1])
            elif len(prop_data)==0:
                prop_des.append('-')
            else:
                prop_des.append(prop_data[1].text.strip().split("Property Description")[1])
        except AttributeError:
            prop_des.append("-") 
        except IndexError:
            prop_des.append("-")
                    
        # getting the facilities data and extracting the text into a list. Incase of missing data "-" is appended.        
        try:
            facilities_data= soup.find("ul",class_='facility-sections')
            facilities.append(facilities_data.text.strip().replace("t_","").lower().replace("facilitycategoryfree   ","").replace("facilitycategorygeneral   ","").replace("facilitycategoryservices   ","").replace("facilitycategoryfoodanddrink   ","").replace("facilitycategoryentertainment   ","").strip())
        except AttributeError:
            facilities.append("-")


def scrap_hostel_world(url):
    
    # opening the target website.
    driver.get(url)
    # putting the program on hold for 3 seconds so that the page gets loaded properly.
    time.sleep(3)
    
    # using the explicit wait method till the search field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div")))
    # finding the webelement of the search field and clicking on in so that it gets activated for input.
    srch_activate= driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div").click()
    
    # using the explicit wait method till the activated-search-field is loaded.
    wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input")))
    # finding the webelement for the activated-search-field and sending the required keys.
    srch= driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/div/input")
    srch.send_keys("London")

    try:
        # using the explicit wait method till the "London" result is loaded in recommended results.
        wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div")))
        # finding the webelement for the first result and clicking on it.
        driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div").click()
        # finding the webelement for the search button and clicking on it.
        btn= driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div[2]/div/div[5]/button").click()
    except TimeoutException:
        # if no reults are loaded in recommended results then print a message and return the function.
        print("No Results Found!!!")
        return
    
    # defining a variable to keep track of the page it is processing.
    count=1
    print(f'Processing Page{count}...')
    # calling the function "get_data" to scrap the first page results.
    get_data()
    
    #running a for loop for subsequent pages where we keep clicking on the next-page button till the button becomes disabled and we face timeout-exception. Then we break the loop.
    for i in range(5):
        try:
            wait= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,"//div[@class='pagination-item pagination-next']")))
            driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH,"//div[@class='pagination-item pagination-next']"))
            count+=1
        except TimeoutException:
            break        
        time.sleep(1)
        print(f'Processing Page{count}...')
        get_data()
    
    # the final dataframe.
    hostels= pd.DataFrame({ "Name":names,
                            "Distance from City-Center":dist_cc,
                            "Rating":ratings, 
                            "Dorms from Price(₹)":dorm_prices, 
                            "Privates from Price(₹)":pvt_prices,
                            "Overall Review":overall_reviews,
                            "Total Reviews":total_reviews,
                            "Facilities":facilities, 
                            "Property Description": prop_des                        
                           })
    # returning the dataframe.
    return hostels

# connecting to the webdriver.
driver= webdriver.Chrome("chromedriver.exe")

# mentioning the target website.
url= "https://www.hostelworld.com/"

# calling the scraping function and saving the results.
df= scrap_hostel_world(url)

#closing the webdriver.
driver.close()
df

Processing Page1...
Processing Page2...


,Name,Distance from City-Center,Rating,Dorms from Price(₹),Privates from Price(₹),Overall Review,Total Reviews,Facilities,Property Description
0,St Christopher's Village,1.8km,8.1,1724,No Privates Available,Fabulous,11400 Total Reviews,linenincluded freecitymaps free wifi freein...,COVID 19 Policy Update.\nIn response to Coro...
1,Selina Camden,5.5km,8.5,8500,27466,Fabulous,34 Total Reviews,linenincluded towelsincluded free wifi loc...,"Among underground music venues, innovative m..."
2,Wombat's City Hostel London,3.6km,9.1,2857,18617,Superb,13875 Total Reviews,linenincluded freecitymaps free wifi freein...,A safe haven in the middle of the metropolis...
3,Generator London,3km,7.7,2394,8808,Very Good,7067 Total Reviews,linenincluded freecitymaps free wifi freein...,Generator London is a design hotel-hostel lo...
4,Urbany Hostel London,5.4km,9.3,3830,14365,Superb,419 Total Reviews,linenincluded free wifi lockers microwave ...,"Welcome to Urbany Hostel London, our first i..."
5,Safestay London Elephant & Castle,1.7km,7.0,1596,15959,Very Good,4412 Total Reviews,linenincluded freecitymaps free wifi locke...,Safestay at Elephant & Castle is ideal if yo...
6,No.8 Willesden Hostel London,10km,6.8,995,No Privates Available,Good,4805 Total Reviews,linenincluded free wifi lockers keycardacc...,At No.8 we take pride in our Customer Servic...
7,No.8 Seven Sisters,9km,6.2,946,No Privates Available,Good,3824 Total Reviews,linenincluded free wifi lockers keycardacc...,"For Great Facilities, Comfort & Affordabilit..."
8,London Backpackers,11.9km,8.1,1596,No Privates Available,Fabulous,4294 Total Reviews,linenincluded freecitymaps free wifi freein...,show more
9,Barmy Badger Backpackers,5.5km,9.0,2869,8414,Superb,1793 Total Reviews,breakfastincluded linenincluded freecitymaps...,The Badger is unique for London in that we h...
